## Deribit API Request  - For BTC and ETH Options Data
#### ALL Options data 

### Specfic Settlement Period ( Day, Week, Month) 

In [ ]:
import pandas as pd
import json
import requests
import re
from datetime import datetime
from tqdm import tqdm
import concurrent.futures
import time 

# Functions
def get_option_name_and_settlement(coin):
    """
    :param coin: crypto-currency coin name ('BTC', 'ETH')
    :return: 2 lists:
                        1. list of traded options for the selected coin;
                        2. list of settlement period for the selected coin.
    """
    r = requests.get("https://test.deribit.com/api/v2/public/get_instruments?currency=" + coin + "&kind=option")
    result = json.loads(r.text)
    # Get option name
    name = pd.json_normalize(result['result'])['instrument_name']
    name = list(name)

    # Get option settlement period
    settlement_period = pd.json_normalize(result['result'])['settlement_period']
    settlement_period = list(settlement_period)

    return name, settlement_period 

def fetch_option_data(option_name):
    """Fetch the option data for a given option name with a small delay to avoid rate limiting."""
    time.sleep(0.1)  # Add a short delay to avoid hitting rate limits
    r = requests.get(f'https://test.deribit.com/api/v2/public/get_order_book?instrument_name={option_name}')
    result = json.loads(r.text)
    return pd.json_normalize(result['result'])


def extract_details(instrument_name, coin):
    """
    Extract expiration date, strike price, and option type (call or put) from instrument name.
    Adjusts to include selected coin (e.g., 'BTC' or 'ETH').
    """
    match = re.match(fr"{coin}-(\d+[A-Z]{{3}}\d+)-(\d+)-([CP])", instrument_name)
    if match:
        expiration_date = match.group(1)
        strike_price = match.group(2)
        option_type = 'Call' if match.group(3) == 'C' else 'Put'
        return expiration_date, strike_price, option_type
    return None, None, None



def get_option_data(coin, settlement_per):
    """
    :param coin: crypto-currency coin name ('BTC', 'ETH')
    :return: pandas DataFrame with all option data for a given coin, filtered for options with specified settlement.
    """
    # Get option name and settlement
    coin_name, settlement_period = get_option_name_and_settlement(coin)

    # Filter options that have the specified settlement period
    coin_name_filtered = [coin_name[i] for i in range(len(coin_name)) if settlement_period[i] == settlement_per]

    # Initialize progress bar
    pbar = tqdm(total=len(coin_name_filtered))

    # Fetch data concurrently using ThreadPoolExecutor
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_to_option = {executor.submit(fetch_option_data, name): name for name in coin_name_filtered}
        coin_df = []
        
        for future in concurrent.futures.as_completed(future_to_option):
            try:
                data = future.result()
                data['settlement_period'] = settlement_per
                coin_df.append(data)
            except Exception as exc:
                print(f'Error fetching data: {exc}')
            pbar.update(1)

    # Finalize DataFrame
    if len(coin_df) > 0:
        coin_df = pd.concat(coin_df)

    # Remove unnecessary columns
    columns = ['state', 'estimated_delivery_price']
    if not coin_df.empty:
        coin_df.drop(columns, inplace=True, axis=1)

    #coin_df['Expiration Date'], coin_df['Strike Price'] = zip(*coin_df['instrument_name'].apply(lambda x: extract_details(x, coin)))

    coin_df['Expiration Date'], coin_df['Strike Price'], coin_df['Option Type'] = zip(*coin_df['instrument_name'].apply(lambda x: extract_details(x, coin)))
    
    today = datetime.today()
    coin_df['Time to Expiration'] = coin_df['Expiration Date'].apply(lambda x: (datetime.strptime(x, '%d%b%y') - today).days / 365 if x else None)

    coin_df.to_csv('data/data.csv', index=False)
 
    pbar.close()

    return coin_df


In [18]:
data = get_option_data('ETH','week')
data=data[["instrument_name", "Option Type", 
           'mark_price', 'underlying_price', 'mark_iv', 
           'greeks.vega', 'Expiration Date', 'Strike Price', 'Time to Expiration']]

data['Strike Price'] = pd.to_numeric(data['Strike Price'], errors='coerce').astype('float64')

 20%|██        | 33/164 [00:21<00:34,  3.80it/s]

Error fetching data: HTTPSConnectionPool(host='test.deribit.com', port=443): Max retries exceeded with url: /api/v2/public/get_order_book?instrument_name=ETH-1NOV24-2400-P (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000002635DB0F0E0>, 'Connection to test.deribit.com timed out. (connect timeout=None)'))
Error fetching data: HTTPSConnectionPool(host='test.deribit.com', port=443): Max retries exceeded with url: /api/v2/public/get_order_book?instrument_name=ETH-1NOV24-2100-C (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000002635DB0DD90>, 'Connection to test.deribit.com timed out. (connect timeout=None)'))
Error fetching data: HTTPSConnectionPool(host='test.deribit.com', port=443): Max retries exceeded with url: /api/v2/public/get_order_book?instrument_name=ETH-1NOV24-2600-P (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000002635DB0E5D0>, 'Connection to test.deribit.com timed out. (connect 

 33%|███▎      | 54/164 [00:22<00:06, 15.81it/s]

Error fetching data: HTTPSConnectionPool(host='test.deribit.com', port=443): Max retries exceeded with url: /api/v2/public/get_order_book?instrument_name=ETH-1NOV24-2800-P (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000002635DCD6F60>, 'Connection to test.deribit.com timed out. (connect timeout=None)'))


 96%|█████████▋| 158/164 [00:35<00:00, 12.51it/s]C:\Users\Ethan Vaz Falcao\AppData\Local\Temp\ipykernel_19096\1825986759.py:83: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  coin_df = pd.concat(coin_df)
100%|██████████| 164/164 [00:35<00:00,  4.56it/s]
C:\Users\Ethan Vaz Falcao\AppData\Local\Temp\ipykernel_19096\734503506.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Strike Price'] = pd.to_numeric(data['Strike Price'], errors='coerce').astype('float64')


In [19]:
data.isnull().sum()

instrument_name       0
Option Type           0
mark_price            0
underlying_price      0
mark_iv               0
greeks.vega           0
Expiration Date       0
Strike Price          0
Time to Expiration    0
dtype: int64

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 145 entries, 0 to 0
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   instrument_name     145 non-null    object 
 1   Option Type         145 non-null    object 
 2   mark_price          145 non-null    float64
 3   underlying_price    145 non-null    float64
 4   mark_iv             145 non-null    float64
 5   greeks.vega         145 non-null    float64
 6   Expiration Date     145 non-null    object 
 7   Strike Price        145 non-null    float64
 8   Time to Expiration  145 non-null    float64
dtypes: float64(6), object(3)
memory usage: 11.3+ KB


In [16]:
data.to_csv('data/data.csv', index=False)